In [1]:
# Cell 1: Install & Auth (Google AI Studio Version)
# ==============================================================================
print("📦 Installing dependencies...")
# We switch from 'vertexai' to the simpler 'google-generativeai' library
%pip install -q google-generativeai pydantic

import os
from google.colab import userdata
import google.generativeai as genai

try:
    print("🔑 Authenticating with API Key...")
    # 1. Fetch Key from Secrets
    api_key = userdata.get('GOOGLE_API_KEY')

    # 2. Configure the library
    genai.configure(api_key=api_key)

    print("✅ Success! Authenticated with Google AI Studio.")

except Exception as e:
    print(f"❌ Auth Failed: {e}")
    print("Did you create a secret named 'GOOGLE_API_KEY' in the 🔑 tab?")

📦 Installing dependencies...
🔑 Authenticating with API Key...
✅ Success! Authenticated with Google AI Studio.


In [9]:
!ls

adhd-assistant.ipynb  LICENSE	   README.md	     setup_config.py
agents.py	      __pycache__  requirements.txt  tools.py


In [10]:
%%writefile agents.py
"""Core agent definitions for the ADHD assistant architecture.

This module defines three collaborating agents:
1. ConversationManagerAgent: orchestrates the flow.
2. TaskLogicAgent: decomposes user intent (Using Google AI Studio / Free Tier).
3. ToolExecutionAgent: prepares and executes tool calls.
"""

from __future__ import annotations

import os
import json
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional

import google.generativeai as genai
from tools import execute_tool

# ---- Shared data models --------------------------------------------------------------------

@dataclass
class TaskItem:
    description: str
    status: str = "pending"
    due: Optional[str] = None
    priority: Optional[str] = None
    conflicts: List[str] = field(default_factory=list)

@dataclass
class TaskPlan:
    tasks: List[TaskItem]
    encouragement: Optional[str] = None
    conflicts: List[str] = field(default_factory=list)

@dataclass
class ToolAction:
    kind: str
    payload: Dict[str, Any]
    description: str

@dataclass
class AgentTurn:
    user_facing_message: str
    tasks: List[TaskItem] = field(default_factory=list)
    pending_actions: List[ToolAction] = field(default_factory=list)
    requires_confirmation: bool = True

# ---- Agents --------------------------------------------------------------------------------

class TaskLogicAgent:
    """The engine: decomposes user intent using the Free Tier API."""

    def __init__(self, model_name: str = "gemini-1.5-pro-002"): # <--- CHANGED to valid model name
        self.model = genai.GenerativeModel(model_name)

    def decompose_brain_dump(
        self, user_text: str, context: Optional[Dict[str, Any]] = None
    ) -> TaskPlan:
        context = context or {}

        prompt = self._construct_prompt(user_text, context)

        try:
            response = self.model.generate_content(
                prompt,
                generation_config=genai.GenerationConfig(
                    response_mime_type="application/json",
                    temperature=0.2
                )
            )
            plan = self._parse_model_response(response)
        except Exception as e:
            print(f"Error calling model or parsing response: {e}")
            plan = TaskPlan(
                tasks=[TaskItem(description=user_text)],
                conflicts=["I had trouble decomposing that. Could you list them one by one?"]
            )

        if context.get("encouragement_override"):
            plan.encouragement = context.get("encouragement_override")

        return plan

    def _construct_prompt(self, user_text: str, context: Dict[str, Any]) -> str:
        user_preferences = context.get("user_preferences", "No specific preferences.")

        return f"""
        You are an expert Task Decomposer for ADHD assistants.
        GOAL: Break down the user's text into atomic tasks.

        OUTPUT SCHEMA (JSON):
        {{
            "reasoning": "Step-by-step analysis string",
            "tasks": [
                {{
                    "description": "Short task description",
                    "due": "Due date or null",
                    "priority": "high/medium/low"
                }}
            ],
            "conflicts": ["List of potential conflicts strings"],
            "encouragement": "Encouraging message string"
        }}

        --- USER CONTEXT ---
        {user_preferences}
        --------------------

        User's Brain Dump:
        "{user_text}"
        """

    @staticmethod
    def _parse_model_response(response) -> TaskPlan:
        try:
            response_text = response.text
            response_dict = json.loads(response_text)
        except Exception:
            return TaskPlan(tasks=[], conflicts=["Model response error"])

        tasks = [TaskItem(**task_data) for task_data in response_dict.get("tasks", [])]
        return TaskPlan(
            tasks=tasks,
            conflicts=response_dict.get("conflicts", []),
            encouragement=response_dict.get("encouragement", "You got this!"),
        )


class ToolExecutionAgent:
    """The hands: schedules tasks, sets reminders."""

    def propose_actions(self, tasks: List[TaskItem]) -> List[ToolAction]:
        actions: List[ToolAction] = []
        for task in tasks:
            if task.due:
                actions.append(
                    ToolAction(
                        kind="schedule_event",
                        payload={"task_description": task.description, "due_date": task.due, "priority": task.priority or 'normal'},
                        description=f"✅ Schedule '{task.description}' for {task.due}",
                    )
                )
            else:
                actions.append(
                    ToolAction(
                        kind="set_reminder",
                        payload={"task_description": task.description, "remind_at": '1 hour from now'},
                        description=f"🔔 Set reminder for '{task.description}'",
                    )
                )
        return actions

    def execute_actions(self, actions: List[ToolAction]) -> List[Any]:
        results: List[Any] = []
        for action in actions:
            try:
                result = execute_tool(action.kind, action.payload)
                results.append(result)
            except Exception as e:
                print(f"Error executing action '{action.kind}': {e}")
                results.append({"status": "error", "details": str(e)})
        return results


class ConversationManagerAgent:
    """The face/orchestrator."""

    def __init__(self, task_agent: TaskLogicAgent, tool_agent: ToolExecutionAgent) -> None:
        self.task_agent = task_agent
        self.tool_agent = tool_agent

    def handle_user_message(
        self,
        user_text: str,
        user_id: str = "default_user",
        auto_confirm: bool = False,
    ) -> AgentTurn:

        context_action = ToolAction(
            kind="get_user_context",
            payload={"user_id": user_id},
            description="Fetching user context.",
        )
        context_result = self.tool_agent.execute_actions([context_action])
        context = context_result[0].get("context", {})

        plan = self.task_agent.decompose_brain_dump(user_text=user_text, context=context)

        pending_actions = self.tool_agent.propose_actions(plan.tasks)
        requires_confirmation = not auto_confirm

        if auto_confirm:
            self.tool_agent.execute_actions(pending_actions)

        message_parts: List[str] = []
        if plan.encouragement:
            message_parts.append(plan.encouragement)

        if plan.tasks:
            message_parts.append("\nHere's what I've broken down for you:")
            for idx, task in enumerate(plan.tasks, start=1):
                task_details = [f"{idx}. {task.description}"]
                if task.due:
                    task_details.append(f" (Due: {task.due})")
                message_parts.append("".join(task_details))

        if pending_actions and requires_confirmation:
            message_parts.append("\nI'll set these up for you:")
            for action in pending_actions:
                message_parts.append(f"- {action.description}")
            message_parts.append("\nSound good?")
        elif not plan.tasks:
            message_parts.append("I couldn't find any specific tasks to list. Could you rephrase?")

        return AgentTurn(
            user_facing_message="\n".join(message_parts),
            tasks=plan.tasks,
            pending_actions=pending_actions,
            requires_confirmation=requires_confirmation,
        )

Overwriting agents.py


In [3]:
import os

# 1. Navigate into the project folder
if os.path.exists("adhd-assistant-capstone"):
    %cd adhd-assistant-capstone
    print("📂 Navigated to project folder.")
else:
    print("⚠️ Folder not found. Cloning now...")
    !git clone https://github.com/viveksahukar/adhd-assistant-capstone.git
    %cd adhd-assistant-capstone

# 2. Re-create the missing setup_config.py file
# (We do this to ensure it exists even if it wasn't pushed to GitHub)
with open("setup_config.py", "w") as f:
    f.write('''
import os
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud import aiplatform as vertexai
import json

def initialize_environment(project_id: str):
    print("--- 🚀 Starting Cloud-Native Authentication ---")

    try:
        # 1. Get the JSON string from Colab Secrets
        # Make sure the name inside get() matches your Secret name exactly!
        key_json = userdata.get('GCP_CREDENTIALS')

        # 2. Convert that string into a format Google's auth library understands
        key_info = json.loads(key_json)
        credentials = service_account.Credentials.from_service_account_info(key_info)

        # 3. Initialize Vertex AI
        os.environ["GCP_PROJECT_ID"] = project_id
        vertexai.init(
            project=project_id,
            location="us-central1",
            credentials=credentials
        )

        print("✅ Success! Authenticated using Colab Secrets.")
        print(f"Service Account: {credentials.service_account_email}")

    except Exception as e:
        print(f"❌ Auth Failed: {e}")
        print("Check: Did you create a secret named 'GCP_CREDENTIALS' in the 🔑 tab?")

    return None, os, vertexai
''')
print("✅ setup_config.py created/verified.")

# 3. Install Dependencies
print("📦 Installing dependencies...")
%pip install -q -r requirements.txt

# 4. Run your Initialization
print("🚀 Initializing Project...")
from setup_config import initialize_environment

# Replace with your project ID
PROJECT_ID = "adhd-assistant-capstone"
auth, os, vertexai = initialize_environment(PROJECT_ID)

⚠️ Folder not found. Cloning now...
Cloning into 'adhd-assistant-capstone'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 70 (delta 37), reused 37 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 34.18 KiB | 972.00 KiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/adhd-assistant-capstone
✅ setup_config.py created/verified.
📦 Installing dependencies...
🚀 Initializing Project...
--- 🚀 Starting Cloud-Native Authentication ---
✅ Success! Authenticated using Colab Secrets.
Service Account: colab-runner@adhd-assistant-capstone.iam.gserviceaccount.com


In [4]:
# Cell 2: Project Initialization
# =================================================================
from setup_config import initialize_environment

# Your specific Project ID
PROJECT_ID = "adhd-assistant-capstone"

# Initialize Vertex AI and auth
# This returns the modules if you need to inspect them, but mainly sets up the global state.
auth, os, vertexai = initialize_environment(PROJECT_ID)

--- 🚀 Starting Cloud-Native Authentication ---
✅ Success! Authenticated using Colab Secrets.
Service Account: colab-runner@adhd-assistant-capstone.iam.gserviceaccount.com


In [5]:
# Cell 3: Import Agent Modules
# =================================================================
try:
    from agents import ConversationManagerAgent, TaskLogicAgent, ToolExecutionAgent
    print("✅ Successfully imported Agent classes.")
except ImportError as e:
    print(f"❌ Error importing agents: {e}")
    print("Make sure agents.py and tools.py are in the current folder.")

✅ Successfully imported Agent classes.


In [6]:
# Cell 4: Create Agent Instances
# =================================================================
# 1. Create the Specialist Agents
# Note: TaskLogicAgent now defaults to 'gemini-2.5-pro' per our refinement
task_agent = TaskLogicAgent()
tool_agent = ToolExecutionAgent()

# 2. Create the Supervisor (Conversation Manager)
# This injects the specialists into the coordinator
manager = ConversationManagerAgent(task_agent=task_agent, tool_agent=tool_agent)

print("✅ Agent System Online. Ready for user input.")

✅ Agent System Online. Ready for user input.


In [7]:
# Cell 5: Simulate User Interaction
# =================================================================

# 1. Define the User's "Brain Dump"
user_input = (
    "I have a big presentation on Friday morning at 10 AM. "
    "I need to call the doctor sometime next week to make an appointment, "
    "and I also have to remember to buy eggs and bread today."
)

print(f"👤 USER SAYS:\n{user_input}\n")
print("-" * 60)

# 2. Handle the message (with auto_confirm=False to test HITL)
# This mimics the "Think" step where the agent plans but waits for approval
agent_turn = manager.handle_user_message(
    user_text=user_input,
    user_id="notebook_test_user_01",
    auto_confirm=False
)

# 3. Display the Agent's proposed response
print(f"🤖 AGENT RESPONDS:\n{agent_turn.user_facing_message}")

👤 USER SAYS:
I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.

------------------------------------------------------------
--- TOOL: Fetching context for user: 'notebook_test_user_01' ---


Error calling model or parsing response: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
🤖 AGENT RESPONDS:

Here's what I've broken down for you:
1. I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.

I'll set these up for you:
- 🔔 Set reminder for 'I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.'

Sound good?


In [8]:
# Cell 6: User Confirmation & Tool Execution
# =================================================================

if agent_turn.requires_confirmation:
    print("✅ USER ACTION: Confirming plan...\n")

    # Execute the pending actions using the Tool Execution Agent
    # This mimics the "Act" step
    results = tool_agent.execute_actions(agent_turn.pending_actions)

    print("--- 🛠️ Tool Execution Results ---")
    for res in results:
        # Check for success/failure in the tool output
        status = res.get("status", "unknown")
        details = res.get("details", str(res))
        print(f"[{status.upper()}] {details}")

else:
    print("ℹ️ No actions required confirmation.")

✅ USER ACTION: Confirming plan...

--- TOOL: Setting reminder: 'I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.' at 1 hour from now ---
--- 🛠️ Tool Execution Results ---
[SUCCESS] Reminder for 'I have a big presentation on Friday morning at 10 AM. I need to call the doctor sometime next week to make an appointment, and I also have to remember to buy eggs and bread today.' set for 1 hour from now.
